## Load Best Model and Execute Configurations

Before loading the model, ensure to execute the following cell containing the necessary classes and model configurations. This step initializes the required components for the model.

In [1]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.3 MB/s eta 0:00:00


In [2]:
import os
import random
import tensorflow as tf
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import pickle
from tensorflow.keras.preprocessing import image
from tensorflow import keras
from keras.layers import (
    GlobalAveragePooling2D, Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization,
    Dropout, Attention, ReLU, Add, Input, Multiply, Layer
)
from tensorflow.keras.utils import plot_model
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop, SGD
from keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
import shutil
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.metrics import SparseCategoricalAccuracy
import wandb
from tensorflow.keras.callbacks import Callback, ModelCheckpoint


In [3]:


@keras.saving.register_keras_serializable()
class SEBlock(Layer):
  """
    Squeeze-and-Excitation (SE) Block layer.

    Args:
        channels (int): Number of input channels.
        ratio (int): Reduction ratio for the intermediate channels.

    Returns:
        Output tensor after applying the SE block.
  """
  def __init__(self, channels, ratio= 16):
     super(SEBlock, self).__init__()

     self.pooling =  GlobalAveragePooling2D()
     self.fc1 = Dense(channels // ratio, activation='relu')
     self.fc2 = Dense(channels, activation='sigmoid')

     def call(sekf, inputs):
      output = self.pooling(inputs)
      output = self.fc1
      output = self.fc2
      return output

@keras.saving.register_keras_serializable()
class SEResBlock(Layer):
    """
    Forward pass of the SE Block layer.

    Args:
        inputs: Input tensor.

    Returns:
        Output tensor after applying the SE block.
    """
    def __init__(self, filters):
        super(SEResBlock, self).__init__()

        # First convolution block in residual
        self.conv1 = Conv2D(filters, (3, 3), padding='same', strides=1)
        self.bn1 = BatchNormalization()
        self.relu1 = ReLU()

        # Second convolution block in residual
        self.conv2 = Conv2D(filters, (3, 3), padding='same', strides=1)
        self.bn2 = BatchNormalization()

        # SE Block
        self.se_block = SEBlock(filters)

        # Shortcut connection
        self.shortcut = Conv2D(filters, (1, 1), padding='same', strides=1)
        self.add = Add()
        self.relu2 = ReLU()

    def call(self, inputs):
        """
        Forward pass of the SERes Block layer.

        Args:
            inputs: Input tensor.

        Returns:
            Output tensor after applying the SERes block.
        """
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.bn2(x)

        # Apply SE Block
        se_output = self.se_block(x)
        x = Multiply()([x, se_output])

        shortcut = self.shortcut(inputs)
        x = self.add([x, shortcut])
        x = self.relu2(x)

        return x


@keras.saving.register_keras_serializable()
class SEResShallowV1(tf.keras.Model):
    """
    Shallow ResNet model with Squeeze-and-Excitation (SE) blocks.

    Args:
        num_classes (int): Number of output classes.
        filters (int): Number of filters in the initial convolutional layer.
        num_blocks (int): Number of residual blocks.

    Returns:
        Output tensor representing class probabilities.
    """
    def __init__(self, num_classes=5, filters=32, num_blocks=2):
        super(SEResShallowV1, self).__init__()

        # Initial convolution block
        self.conv1 = Conv2D(filters, (3, 3), padding='same', strides=1)
        self.bn1 = BatchNormalization()
        self.relu1 = ReLU()

        # Residual blocks
        self.res_blocks = [SEResBlock(filters*pow(2, i)) for i in range(num_blocks)]

        # Global average pooling and final dense layer
        self.global_avg_pooling = GlobalAveragePooling2D()
        self.fc = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        """
        Forward pass of the SEResShallowV1 model.

        Args:
            inputs: Input tensor.

        Returns:
            Output tensor representing class probabilities.
        """
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu1(x)
        for res_block in self.res_blocks:
            x = res_block(x)

        x = self.global_avg_pooling(x)
        x = self.fc(x)

        return x

def create_directory_or_file(path, is_dir=True):
    """
    Create a directory or file at the specified path.

    Args:
        path (str): Path to the directory or file.
        is_dir (bool): If True, create a directory. If False, create a file.

    Returns:
        bool: True if the directory or file already exists, False if it was created.
    """
    if is_dir:
        # Check if the directory exists
        if not os.path.exists(path) or not os.path.isdir(path):
            os.makedirs(path)
            print(f"The directory '{path}' has been created.")
            return False
        return True
    else:
        # Check if the file exists
        if not os.path.exists(path):
            # Create the file if it doesn't exist
            with open(path, 'w') as file:
                pass
            print(f"The file '{path}' has been created.")
            return False
        return True



def save_checkpoint(path, history):
  """ The output directory structure
  output :
    - model_directory
      1. checkpoints
        * model_checkpoint_epoch{number}
        ...
      2. history.pkl
      3. configs.json

  """
  # create checkpoint path if it doesn't exist
  full_path = os.path.join(path, "checkpoints")
  create_directory_or_file(full_path)


  # save current evaluation metrics and loss in history.pkl (create it if it doesn't exist)
  history_path = os.path.join(path, "history.pkl")
  res = create_directory_or_file(history_path, dir = False)

  # if the file doesn't exist, we fill it with
  data = dict()
  if not res :
    index = 0
    data[index] = history

    with open(history_path, 'wb') as file:
      pickle.dump(data, file)

  else:
    try:
      # Open the file for reading in binary mode
      with open(history_path, 'rb+') as file:
        # Load the pickled data from the file
        loaded_data = pickle.load(file)

        # Add the new data inside
        index = max(loaded_data.keys())+1
        loaded_data[index] = history

      with open(history_path, 'wb') as file:
        # Save the new content in the file
        pickle.dump(loaded_data, file)

    except FileNotFoundError:
        print(f"The file '{history_path}' does not exist.")
    except Exception as e:
        print(f"An error occurred: {e}")




class SaveCallback(Callback):
    """
    Callback to save model checkpoints and logs at the end of each epoch.

    Args:
        save_path (str): Path to the directory where checkpoints and logs will be saved.
    """
    def __init__(self, save_path):
        super(SaveCallback, self).__init__()
        self.save_path = save_path

    def on_epoch_end(self, epoch, logs=None):
        # Access the current loss and accuracy
        save_checkpoint(path = self.save_path, history=logs)
        wandb.log(logs)

# Define model configurations
optimizer_configs = {"adam" : {
                        "learning_rate": 0.0005,
                        "beta_1": 0.9,
                        "beta_2": 0.999,
                        "epsilon": 1e-07
                      },

                     "rms_prop": {
                         "learning_rate": 0.001,
                         "rho": 0.9,
                         "momentum": 0.0,
                         "epsilon": 1e-07
                     },

                     "sgd": {
                         "learning_rate": 0.01,
                         "momentum": 0.0
                     }
                  }

loss_configs = {"sparse_categorical_crossentropy":{
               "from_logits": False}}


# Define compiling configurations
model_configs = {"optimizers": optimizer_configs,
           "losses": loss_configs}



## Generate CSV File

This function takes the path to your test data as input and generates a CSV file as output. The CSV file includes the following columns:

- `image_name`: Name of the image file.
- `relative_path`: Relative path to the image file within the provided test data directory.
- `class_label`: Class label assigned to the image.

To use this function, provide the path to your test data directory, and the output CSV file will be created with the specified columns. The CSV file will contain information about each image, making it suitable for further analysis or evaluation.


In [ ]:


# Test images in the directory
def generate_csv_file(directory_path,model_path, random_seed=None, output_csv_path='output.csv'):
    if random_seed is not None:
        random.seed(random_seed)

    all_images = [filename for filename in os.listdir(directory_path) if filename.endswith(('.jpg', '.jpeg', '.png'))]

    predictions = []
    image_names = []
    relative_paths = []
    # Open the file for reading in binary mode


    model = tf.keras.models.load_model(model_path, custom_objects={"se_block": SEBlock, "se_res_block": SEResBlock,"se_res_shallowv1": SEResShallowV1})


    for filename in all_images:
        image_path = os.path.join(directory_path, filename)
        relative_path = os.path.relpath(image_path, directory_path)

        # You may need to adjust this part based on your model input requirements
        # Read the image file and perform minimal processing
        imag = image.load_img(image_path, target_size=(28, 28), color_mode='grayscale')
        imag_arr = image.img_to_array(imag)
        imag_arr = np.expand_dims(imag_arr, axis=0)
        imag = imag_arr / 255.0  # Normalize if needed


        # Make predictions using your model
        prediction = model.predict(imag)
        class_label = np.argmax(prediction)

        predictions.append(class_label)
        image_names.append(filename)
        relative_paths.append(relative_path)

    df = pd.DataFrame({'image_name': image_names, 'relative_path': relative_paths, 'class_label': predictions})
    df.to_csv(output_csv_path, index=False)
    print(f'CSV file saved at {output_csv_path}')


In [ ]:
# Change this path according to your model.keras's directory
model_path = "/content/drive/MyDrive/ISIMA2/ISIMA/Deep learning/Challenge/Challenge/livrable/model.keras"
# Change this path according to your test_data's directory
directory_path = '/content/drive/MyDrive/ISIMA2/ISIMA/Deep learning/Challenge/Challenge/test_data'


generate_csv_file(directory_path, model_path)